# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis, tol=tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770409985184                   -0.52    9.0
  2   -2.771691771728       -2.89       -1.32    1.0
  3   -2.771713924257       -4.65       -2.37    2.0
  4   -2.771714715017       -6.10       -3.94    2.0
  5   -2.771714715243       -9.65       -4.75    2.0


-0.0001334590385656934

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε, tol=tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770164388495                   -0.53    8.0
  2   -2.771767700041       -2.79       -1.29    1.0
  3   -2.771802010390       -4.46       -2.78    2.0
  4   -2.771802073270       -7.20       -4.07    2.0
  5   -2.771802074445       -8.93       -4.66    2.0


0.017610317546926175

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001334590385656934
Displaced dipole:  0.017610317546926175
Polarizability :   1.7743776585491868


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; ρ=res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493947471591e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766521391412e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.850469002411e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694743325864e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088878658333e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.649520545256e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 4.406571867156e-12
[ Info: GMRES linsolve in iter 1; step 8: normres = 1.117610550526e-13
[ Info: GMRES linsolve in iter 1; finished at step 8: normres = 1.117610550526e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.895465297018e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.121499325230e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 5.279382581658e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 6.797650150398e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 6

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.